In [1]:
from typing import List, Tuple, Optional, Union, Iterable

import plotly.graph_objects as go

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from pytorch3d.transforms import quaternion_apply, random_quaternions, quaternion_multiply, quaternion_invert, axis_angle_to_quaternion, se3_exp_map, se3_log_map, quaternion_to_matrix, matrix_to_quaternion

from e3nn import o3

from diffusion_edf.embedding import NodeEmbeddingNetwork
from diffusion_edf.data import SE3, PointCloud, TargetPoseDemo, DemoSequence, DemoSeqDataset, load_demos, save_demos
from diffusion_edf.preprocess import Rescale, NormalizeColor, Downsample, PointJitter, ColorJitter
from diffusion_edf.wigner import TransformFeatureQuaternion
from diffusion_edf.score_model import ScoreModel

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
unit_len = 0.01
scene_voxel_size = 0.01
grasp_voxel_size = 0.01

scene_voxel_size = scene_voxel_size / unit_len
grasp_voxel_size = grasp_voxel_size / unit_len


rescale_fn = Rescale(rescale_factor=1/unit_len)
recover_scale_fn = Rescale(rescale_factor=unit_len)
normalize_color_fn = NormalizeColor(color_mean = torch.tensor([0.5, 0.5, 0.5]), color_std = torch.tensor([0.5, 0.5, 0.5]))
recover_color_fn = NormalizeColor(color_mean = -normalize_color_fn.color_mean / normalize_color_fn.color_std, color_std = 1 / normalize_color_fn.color_std)


scene_proc_fn = Compose([rescale_fn,
                         Downsample(voxel_size=scene_voxel_size, coord_reduction="average"),
                         normalize_color_fn])
scene_unproc_fn = Compose([recover_color_fn, recover_scale_fn])
grasp_proc_fn = Compose([rescale_fn,
                         Downsample(voxel_size=grasp_voxel_size, coord_reduction="average"),
                         normalize_color_fn])
grasp_unproc_fn = Compose([recover_color_fn, recover_scale_fn])

In [3]:
import math

device = 'cuda:0'
compile = False
eval = True

irreps_input = o3.Irreps('3x0e')
irreps_node_embedding = o3.Irreps('32x0e+16x1e+8x2e') #o3.Irreps('128x0e+64x1e+32x2e')
irreps_sh = o3.Irreps('1x0e+1x1e+1x2e')
fc_neurons = [128, 64, 64]
num_heads = 4
alpha_drop = 0.2
proj_drop = 0.0
drop_path_rate = 0.0
irreps_mlp_mid = 2
n_scales = 4
pool_ratio = 0.5

In [4]:
score_model = ScoreModel(irreps_input = irreps_input,
                         irreps_emb_init = irreps_node_embedding,
                         irreps_sh = irreps_sh,
                         fc_neurons_init = [32, 16, 16],
                         num_heads = 4,
                         n_scales = 4,
                         pool_ratio = 0.5,
                         dim_mult = [1, 2, 3, 4],
                         n_layers = 2,
                         gnn_radius = 2.0,
                         cutoff_radius = 3.0,
                         weight_feature_dim = 20,
                         query_downsample_ratio = 0.3,
                         device=device,
                         deterministic = True,
                         compile_head = compile)

score_model = score_model.to(device)
if eval:
    score_model = score_model.eval()

/home/hw/anaconda3/envs/diff_edf/lib/python3.8/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


# Load demo

In [5]:
optimizer = torch.optim.Adam(list(score_model.parameters()), lr=1e-4, betas=(0.9, 0.98), eps=1e-09, weight_decay=1e-4, amsgrad=True)

In [6]:
loss_fn = torch.nn.MSELoss(reduction='mean')

In [7]:
trainset = DemoSeqDataset(dataset_dir="demo/test_demo", annotation_file="data.yaml", device=device)
train_dataloader = DataLoader(trainset, shuffle=True, collate_fn=lambda x:x)

In [8]:
loss_list = []

In [9]:
optimizer.zero_grad(set_to_none=True)

In [17]:
max_epochs = 10
N_samples = 10

iter = 0
for epoch in range(1, max_epochs+1):
    for train_batch in train_dataloader:
        iter += 1
        assert len(train_batch) == 1, "Batch training is not supported yet."

        optimizer.zero_grad()

        demo_seq: DemoSequence = train_batch[0]
        demo: TargetPoseDemo = demo_seq[1]

        scene_raw: PointCloud = demo.scene_pc
        grasp_raw: PointCloud = demo.grasp_pc
        target_poses: SE3 = demo.target_poses

        scene_proc = scene_proc_fn(scene_raw).to(device)
        grasp_proc = grasp_proc_fn(grasp_raw).to(device)
        target_poses = rescale_fn(target_poses).to(device)

        T_target = target_poses.poses

        disp_q = torch.randn(N_samples, 3, device=device) * 0.5
        disp_x = torch.randn(N_samples, 3, device=device) * 3.0
        disp_T = se3_exp_map(torch.cat([disp_x, disp_q], dim=-1))
        disp_T = torch.cat([matrix_to_quaternion(disp_T[...,:3,:3]), disp_T[...,-1,:3]], dim=-1)


        T = torch.cat([quaternion_multiply(T_target[..., :4], disp_T[..., :4]), 
                    quaternion_apply(T_target[..., :4], disp_T[..., 4:]) + T_target[..., 4:]], dim=-1)


        key_feature = scene_proc.colors
        key_coord = scene_proc.points
        key_batch = torch.zeros(len(key_coord), device=device, dtype=torch.long)
        query_feature = grasp_proc.colors
        query_coord = grasp_proc.points
        query_batch = torch.zeros(len(query_coord), device=device, dtype=torch.long)

        lin_vel, ang_vel = score_model(T=T,
                                       key_feature=key_feature, key_coord=key_coord, key_batch=key_batch,
                                       query_feature=query_feature, query_coord=query_coord, query_batch=query_batch)
        
        loss = loss_fn(torch.cat([ang_vel, lin_vel], dim=-1), torch.cat([disp_q, disp_x], dim=-1))
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    print(torch.tensor(loss_list[-len(trainset):]).mean())

tensor(5.4813)
tensor(4.5744)
tensor(4.1819)
tensor(4.9649)
tensor(4.4160)
tensor(4.4867)
tensor(4.8789)


RuntimeError: cannot reshape tensor of 0 elements into shape [0, 4, -1] because the unspecified dimension size -1 can be any value and is ambiguous

In [ ]:
# import gc
# for obj in gc.get_objects():
#     try:
#         if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#             print(type(obj), obj.size())
#     except:
#         pass